# Immagini naturali e "miniAlexNet"

In [ ]:
import torch
from torchvision.datasets import CIFAR100
from torch.utils.data import DataLoader
from torchvision import transforms
import numpy as np

from torch.optim import SGD
from torch.utils.tensorboard import SummaryWriter
from sklearn.metrics import accuracy_score
from os.path import join

In [ ]:
np.random.seed(1328)
torch.random.manual_seed(1328)

Importo il train classifier e l'avg meter e il test classifier

In [ ]:
class AverageValueMeter():
    def __init__(self):
        self.reset()
    
    def reset(self):
        self.sum = 0
        self.num = 0
    
    def add(self, value, num):
        self.sum += value*num
        self.num += num

    def value(self):
        try:
            return self.sum/self.num
        except:
            return None

In [ ]:
def train_classifier(model, train_loader, test_loader, exp_name='experiment', lr=0.01, epochs=10, momentum=0.99, logdir='logs'):

    criterion = nn.CrossEntropyLoss()
    optimizer = SGD(model.parameters(), lr, momentum=momentum)

    # meters
    loss_meter = AverageValueMeter()
    acc_meter = AverageValueMeter()
    # writer
    writer = SummaryWriter(join(logdir, exp_name))
    # device
    device = "cuda" if torch.cuda.is_available() else "cpu"
    model.to(device)
    ## definiamo un dizionario contenente i loader di training e test
    loader = {
        'train': train_loader,
        'test': test_loader
    }
    global_step = 0
    for e in range(epochs):
        # iteriamo tra due modalità: train e test
        for mode in ['train', 'test']:
            loss_meter.reset(); acc_meter.reset()
            model.train() if mode == 'train' else model.eval()
            with torch.set_grad_enabled(mode=='train'): # abilitiamo i gradienti o solo in training
                for i, batch in enumerate(loader[mode]):
                    x=batch[0].to(device) # portiamoli su device corretto
                    y=batch[1].to(device)
                    output = model(x)

                    # aggiorniamo il global_step
                    # conterrà il numero di campioni visti durante il training
                    n = x.shape[0]  # n di elementi nel batch
                    global_step += n
                    l = criterion(output, y)

                    if mode=='train':
                        l.backward()
                        optimizer.step()
                        optimizer.zero_grad()

                    acc = accuracy_score(y.to('cpu'), output.to('cpu').max(1)[1])
                    loss_meter.add(l.item(), n)
                    acc_meter.add(acc,n)

                    # loggiamo i risultati iterazione per iterazione solo durante il training
                    if mode == 'train':
                        writer.add_scalar('loss/train', loss_meter.value(), global_step=global_step)
                        writer.add_scalar('accuracy/train', acc_meter.value(), global_step=global_step)

                        # una volta finita l'epoca sia nel caso di training che di test loggiamo le stime finali

                writer.add_scalar('loss/' + mode, loss_meter.value(), global_step=global_step)
                writer.add_scalar('accuracy/' + mode, acc_meter.value(), global_step=global_step)

                        ## conserviamo i pesi del modello alla fine di un ciclo di training e test

        torch.save(model.state_dict(), '%s-%d.pth'%(exp_name, e+1))
    return model

In [ ]:
def test_classifier(model, loader):
  device = "cuda" if torch.cuda.is_available() else "cpu"
  model.to(device)
  predictions, labels = [], []
  for batch in loader:
    x = batch[0].to(device)
    y = batch[1].to(device)
    output = model(x)
    preds = output.to('cpu').max(1)[1].numpy()
    labs = y.to('cpu').numpy()
    predictions.extend(list(preds))
    labels.extend(list(labs))
  return np.array(predictions), np.array(labels)

In [ ]:
%load_ext tensorboard
%tensorboard --logdir logs

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6006 (pid 103), started 1:54:23 ago. (Use '!kill 103' to kill it.)

<IPython.core.display.Javascript object>

Carichiamo il dataset CIFRAR-100. Normalizziamo le immagini usando media e varianza per canale che sono state precomputate

In [ ]:
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize( ( 0.4914, 0.4822, 0.4465) , (0.2023, 0.1994, 0.2010 ) )])

cifar100_train = CIFAR100(root='cifar100',train=True, download=True, transform=transform)
cifar100_test = CIFAR100(root='cifar100',train=False, download=True, transform=transform)
cifar100_train_loader = DataLoader(cifar100_train, batch_size=1024, num_workers=2, shuffle=True)
cifar100_test_loader = DataLoader(cifar100_test, batch_size=1024, num_workers=2)


Extracting cifar100/cifar-100-python.tar.gz to cifar100
Files already downloaded and verified


Ridefiniamo il modello per applicarlo su immagini rgb (LeNetV2) per prendere in input immagini 32x32. Aumenteremo il numero di feature maps e unità nei layer fully connected per aumentare le capacità della rete. Sostituiremo l'average Pooling con il MaxPooling e le attivazioni Tanh e ReLu

In [ ]:
from torch import nn
class LeNetColor(nn.Module):
  def __init__(self):
    super(LeNetColor, self).__init__()
    #ridefiniamo il modello utilizzando i moduli sequential.
    #ne definiamo due: un "feature extractor", che estrae le feature maps #e un "classificatore" che implementa i livelly FC
    self.feature_extractor = nn.Sequential(
      nn.Conv2d(3, 18, 5), #Input: 3 x 32 x 32. Ouput: 18 x 28 x 28
      nn.MaxPool2d(2), #Input: 18 x 28 x 28. Output: 18 x 14 x 14
      nn.ReLU(),
      nn.Conv2d(18, 28, 5), #Input 18 x 14 x 14. Output: 28 x 10 x 10
      nn.MaxPool2d(2), #Input 28 x 10 x 10. Output: 28 x 5 x 5
      nn.ReLU()
    )

    self.classifier = nn.Sequential(
        nn.Linear(700, 360), # Input 28 * 5 * 5
        nn.ReLU(),
        nn.Linear(360, 252),
        nn.ReLU(),
        nn.Linear(252, 100)
    )

  def forward(self,x):
    #applichiamo tutte le diverse trasformazioni in cascata
    x = self.feature_extractor(x)
    x = self.classifier(x.view(x.shape[0],-1))
    return x

Alleniamo di seguito il modello:

In [ ]:
lenet_cifar100 = LeNetColor()
lenet_cifar100 = train_classifier(lenet_cifar100, cifar100_train_loader, cifar100_test_loader, 'lenet_cifar100', epochs=150)

Calcoliamo l'accuracy

In [12]:
lenet_cifar100_test_predictions, cifar100_labels_test = test_classifier(lenet_cifar100, cifar100_test_loader)

#format to print just 2 decimals instead of all of them when you print a float

print("Accuracy LeNetColor su CIFAR-100: %0.2f%%" % \
  accuracy_score(cifar100_labels_test, lenet_cifar100_test_predictions))

Accuracy LeNetColor su CIFAR-100: 0.23%


L'accuracy è piuttosto bassa. Proviamo a migliorarla aumentando la capacità del modello. Definiamo un modello più "profondo" con 5 livelli di convoluzione e tre layer fully connected. Inseriremo il max pooling solo tra il primo e secondo livello, il secondo e il terzo, e il quinto e il sesto. Per evitare l'eccessiva riduzione di dimensioni delle mappe di features, specifichiamo un padding pari al ceil della dimensione del kernel fratto 2. Questo farà sì che le convoluzioni non riducano le dimensioni delle mappe di features. Utilizziamo kernel di dimensioni più grandi nei primi layer e più piccoli nei layer successivi. Il modello è vagamente ispirato al modello "AlexNet" proposto da Krizhevsky et al. nel 2013.

# MiniAlexNet

All'interno delle slide viene introdotto passo passo miniAlexnet. Tuttavia per aspettare delle ore per la fase di training viene implementata di seguito la versione finale di MiniAlexNet.

Negli esempi visti nelle slide, quando vi è una grossa differenza tra accuracy di training e di test, è dovuto dall'overfitting e si acuisce quando i modelli contengono molti parametri e i dataset sono piccoli. Oltre alla tecnica di regolarizzazione mediante weight decay, esistono altre tecniche per ridurre l'overfitting quali 
- dropout
- data agumentation
- batch normalization

In [ ]:
class MiniAlexNetV4(nn.Module):
  def __init__(self, input_channels=3, out_classes=100):
    super(MiniAlexNetV4, self).__init__()
    # ridefiniamo il modello utilizzando i moduli sequential, ne definiamo due: un "feature extractor" che estrae le feature maps e un "classificatore" implementa i livelli di FC
    self.feature_extractor = nn.Sequential(
        #Conv1
        nn.Conv2d(input_channels, 16, 5, padding=2) # Input: 3x28x28. Output 16x28x28
        nn.MaxPool2d(2),
        
    )